# Lesson 5: Distributed Scaling

In lesson 4 you've learned how to be as efficient as possible on a single CPU.

Now we want to use multiple CPUs at the same time (horizontal scaling) and optimize efficiency of multiple CPUs.

![image](https://raw.githubusercontent.com/nsmith-/2025-09-15-hsf-india-tutorial-chandigarh/cda03896f0f7d4d195db3fedf00a8754f6774214/img/horizontal-and-vertical-scaling.svg)

## Scaling on a single machine

A single computer may have multiple cores available that we can use to concurrently run software.

#### In python the two ways to parallelize programs are: **_multi-processing_** and **_multi-threading_**.

**Multi-processing** is essentially about running multiple OS processes in parallel, each with its own Python interpreter, memory space, etc.

**Multi-threading**, however, uses a shared memory space and a common Python interpreter to spawn multiple threads that can run tasks concurrently. 
_Caution_: In Python multi-threading is in fact not running programs in parallel because of the Global Interpreter Lock (GIL), only since python 3.14 (free-threaded) this is possible.


### Basic rule of thumb for Python:

**Use multi-processing** when you're program is CPU-bound (i.e. it's constantly calculating things).

**Use multi-threading** when you're program is IO-bound (i.e. it's waiting a long time for loading data or network requests).

In [1]:
import concurrent.futures

In [2]:
%%writefile quadratic_formula_scaling.py
# writing to a separate file, because multiprocessing is buggy in notebooks (see: https://bugs.python.org/issue25053)
import numpy as np
import time

a = np.random.uniform(5, 10, 1_000_000)
b = np.random.uniform(10, 20, 1_000_000)
c = np.random.uniform(-0.1, 0.1, 1_000_000)

def quadratic_formula(task_id):
    time.sleep(task_id / 5.0)  # simulate some task_id-dependent workload
    return (-b + np.sqrt(b**2 - 4*a*c)), task_id  # return task_id for identification


# setup 10 tasks
task_ids = range(10)

Overwriting quadratic_formula_scaling.py


### Python for-loop example

In [3]:
%%timeit -n 1 -r 1

from quadratic_formula_scaling import quadratic_formula, task_ids

for task_id in task_ids:
    output, _ = quadratic_formula(task_id)
    print(f"Task {task_id} completed with output {output[:5]}...")  # print first 5 results of each task

Task 0 completed with output [-0.03423688  0.0506     -0.07397691 -0.04395879 -0.0492483 ]...
Task 1 completed with output [-0.03423688  0.0506     -0.07397691 -0.04395879 -0.0492483 ]...
Task 2 completed with output [-0.03423688  0.0506     -0.07397691 -0.04395879 -0.0492483 ]...
Task 3 completed with output [-0.03423688  0.0506     -0.07397691 -0.04395879 -0.0492483 ]...
Task 4 completed with output [-0.03423688  0.0506     -0.07397691 -0.04395879 -0.0492483 ]...
Task 5 completed with output [-0.03423688  0.0506     -0.07397691 -0.04395879 -0.0492483 ]...
Task 6 completed with output [-0.03423688  0.0506     -0.07397691 -0.04395879 -0.0492483 ]...
Task 7 completed with output [-0.03423688  0.0506     -0.07397691 -0.04395879 -0.0492483 ]...
Task 8 completed with output [-0.03423688  0.0506     -0.07397691 -0.04395879 -0.0492483 ]...
Task 9 completed with output [-0.03423688  0.0506     -0.07397691 -0.04395879 -0.0492483 ]...
9.21 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop ea

### Multi-threading example

In [4]:
%%timeit -n 1 -r 1

from quadratic_formula_scaling import quadratic_formula, task_ids


with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(quadratic_formula, task_id) for task_id in task_ids]

for future in concurrent.futures.as_completed(futures):
    output, task_id = future.result()
    print(f"Task {task_id} completed with output {output[:5]}...")  # print first 5 results of each task

Task 0 completed with output [-0.03423688  0.0506     -0.07397691 -0.04395879 -0.0492483 ]...
Task 5 completed with output [-0.03423688  0.0506     -0.07397691 -0.04395879 -0.0492483 ]...
Task 8 completed with output [-0.03423688  0.0506     -0.07397691 -0.04395879 -0.0492483 ]...
Task 7 completed with output [-0.03423688  0.0506     -0.07397691 -0.04395879 -0.0492483 ]...
Task 1 completed with output [-0.03423688  0.0506     -0.07397691 -0.04395879 -0.0492483 ]...
Task 2 completed with output [-0.03423688  0.0506     -0.07397691 -0.04395879 -0.0492483 ]...
Task 4 completed with output [-0.03423688  0.0506     -0.07397691 -0.04395879 -0.0492483 ]...
Task 6 completed with output [-0.03423688  0.0506     -0.07397691 -0.04395879 -0.0492483 ]...
Task 3 completed with output [-0.03423688  0.0506     -0.07397691 -0.04395879 -0.0492483 ]...
Task 9 completed with output [-0.03423688  0.0506     -0.07397691 -0.04395879 -0.0492483 ]...
1.81 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop ea

### Multi-processing example

In [5]:
%%timeit -n 1 -r 1

from quadratic_formula_scaling import quadratic_formula, task_ids

with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(quadratic_formula, task_id) for task_id in task_ids]

for future in concurrent.futures.as_completed(futures):
    output, task_id = future.result()
    print(f"Task {task_id} completed with output {output[:5]}...")  # print first 5 results of each task

Task 9 completed with output [-0.01850971  0.05290874  0.05529105 -0.01086886  0.14588338]...
Task 7 completed with output [-0.06165524  0.01516968 -0.09811029  0.03078042 -0.04925104]...
Task 1 completed with output [ 0.12665125 -0.0743113  -0.09411886  0.01897715  0.01112827]...
Task 3 completed with output [ 0.10597648  0.0145213  -0.02716122 -0.07412861 -0.06141234]...
Task 5 completed with output [-0.09945009  0.08043627  0.00639675 -0.01837581  0.09017917]...
Task 0 completed with output [ 0.07409236  0.00600725 -0.04949217  0.00908     0.1003149 ]...
Task 2 completed with output [0.11341482 0.02816544 0.06614238 0.01770748 0.06747025]...
Task 6 completed with output [ 0.06449472  0.00216018 -0.03168948 -0.06154289 -0.07528261]...
Task 4 completed with output [ 0.06906914 -0.0009371   0.05279456  0.06236099 -0.09148618]...
Task 8 completed with output [ 0.00633929 -0.03936166  0.02405769  0.06793498  0.04995134]...
2.09 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Multi-threading and multi-processing can speed-up your programs by concurrently executing multiple tasks across multiple threads or processes.

Here, we can see in addition the effect of shared (**multi-threading**) vs individual (**multi-processing**) memory space: 
- **multi-threading**: the NumPy random initialization runs _once_ and the memory is shared between all threads, which is why all outputs are the _same_ (`[ 0.00466771  0.00414385  0.00415603 -0.00520032 -0.00212164]...`)
- **multi-processing**: the NumPy random initialization runs in _each_ process again. There's no shared memory, every process has it's own set of `a`, `b`, and `c` arrays, which is why all outputs are _different_. Initializing the arrays in each process adds an additional runtime overhead, which is why here the multi-processing case is slightly slower.

## Scaling on multiple machines

## Dask

There are different ways to scale to multiple machines in a computing cluster. 

The most classic ways are independent of Python and let you submit any type of program 'job' to a cluster, e.g. using HTCondor or Slurm.

In Python, one solution emerged that allows to submit jobs/tasks to a cluster of workers from a Python program: **Dask**.

### Dask example

In [6]:
import dask.distributed

from quadratic_formula_scaling import quadratic_formula, task_ids

In [7]:
%%timeit -n 1 -r 1

with (
    # multi-threading cluster
    dask.distributed.LocalCluster(n_workers=1, threads_per_worker=10) as cluster,
    # multi-processing cluster
    # dask.distributed.LocalCluster(n_workers=10, threads_per_worker=1) as cluster,
    # mixed cluster
    # dask.distributed.LocalCluster(n_workers=2, threads_per_worker=5) as cluster,
    # ---
    # connect to a Dask cluster
    dask.distributed.Client(cluster) as client,
):
    futures = client.map(quadratic_formula, task_ids)

    for future in dask.distributed.as_completed(futures):
        output, task_id = future.result()
        print(f"Task {task_id} completed with output {output[:5]}...")  # print first 5 results of each task

Task 0 completed with output [ 0.04766715 -0.03581469 -0.02952755  0.02982136  0.09562612]...
Task 1 completed with output [ 0.04766715 -0.03581469 -0.02952755  0.02982136  0.09562612]...
Task 2 completed with output [ 0.04766715 -0.03581469 -0.02952755  0.02982136  0.09562612]...
Task 3 completed with output [ 0.04766715 -0.03581469 -0.02952755  0.02982136  0.09562612]...
Task 4 completed with output [ 0.04766715 -0.03581469 -0.02952755  0.02982136  0.09562612]...
Task 5 completed with output [ 0.04766715 -0.03581469 -0.02952755  0.02982136  0.09562612]...
Task 6 completed with output [ 0.04766715 -0.03581469 -0.02952755  0.02982136  0.09562612]...
Task 7 completed with output [ 0.04766715 -0.03581469 -0.02952755  0.02982136  0.09562612]...
Task 8 completed with output [ 0.04766715 -0.03581469 -0.02952755  0.02982136  0.09562612]...
Task 9 completed with output [ 0.04766715 -0.03581469 -0.02952755  0.02982136  0.09562612]...
2.62 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop ea

### Dask in practice

Dask abstracts the cluster from the user. If there's a dask-scheduler running somewhere, we can connect to it using:

```python
with dask.distributed.Client(address='127.0.0.1:8786') as client: # some IP+port address
    ...
```

This is great, as scientist never have to worry about the cluster infrastructure, with a single lince of code you can connect and run your analysis distributed on multiple workers and optionally using multiple threads per worker.

One cluster in HEP that's commonly used for analysis is: https://coffea.casa/.

### Dask is much more than a distributive interface

Dask allows to define compute graphs of complex workflows. These graphs can then be scheduled automatically by the dask-scheduler such that the most optimal execution of tasks is achieved.

In [8]:
import dask

In [9]:
@dask.delayed  # mark function as delayed/task
def increment(i):
    return i + 1

@dask.delayed  # mark function as delayed/task
def add(a, b):
    return a + b


# define compute graph
a, b = 1, 12
c = increment(a)
d = increment(b)
output = add(c, d)

# visualize compute graph
try:
    output.visualize(rankdir="LR", filename="img/compute_graph_simple.svg")  # left to right
except Exception as e:
    ... # install graphviz and python-graphviz to visualize the graph

# compute the result
print("Result:", output.compute())

There is no layout engine support for "dot"
Perhaps "dot -c" needs to be run (with installer's privileges) to register the plugins?


Result: 15


![image](img/compute_graph_simple.svg)

In [10]:
import numpy as np
import dask.array as da

# use dask arrays instead of Numpy arrays for automatic graph construction
a = da.random.uniform(5, 10, 100_000)
b = da.random.uniform(10, 20, 100_000)
c = da.random.uniform(-0.1, 0.1, 100_000)

def quadratic_formula(a, b, c):
    return (-b + np.sqrt(b**2 - 4*a*c)) / (2*a)

# define compute graph
output = quadratic_formula(a, b, c)

# visualize compute graph
try:
    output.visualize(rankdir="LR", filename="img/compute_graph_quadratic_formula.svg")
    output.visualize(rankdir="LR", filename="img/compute_graph_quadratic_formula__optimized.svg", optimize_graph=True)
except Exception as e:
    ... # install graphviz and python-graphviz to visualize the graph


# compute the result
unoptimized_result = dask.compute(output, optimize_graph=False)[0]
print("Result (unoptimized):", unoptimized_result)

optimized_result = dask.compute(output, optimize_graph=True)[0]
print("Result (optimized):", optimized_result)

assert np.allclose(unoptimized_result, optimized_result)


There is no layout engine support for "dot"
Perhaps "dot -c" needs to be run (with installer's privileges) to register the plugins?


Result (unoptimized): [-0.00575828 -0.00599738  0.00358227 ... -0.0017056   0.00664479
 -0.00026088]
Result (optimized): [-0.00575828 -0.00599738  0.00358227 ... -0.0017056   0.00664479
 -0.00026088]


#### Compute Graph

![image](img/compute_graph_quadratic_formula.svg)

#### Compute Graph _optimized_

![image](img/compute_graph_quadratic_formula__optimized.svg)

In [11]:
def fun(a, b, c):
    return np.mean((-b + np.sqrt(b**2 - 4*a*c)) / (2*a))

In [12]:
%%timeit -n 1 -r 1

a = np.random.uniform(5, 10, 200_000_000)
b = np.random.uniform(10, 20, 200_000_000)
c = np.random.uniform(-0.1, 0.1, 200_000_000)

output = fun(a, b, c)
print("Result:", output)

Result: -9.400685890981767e-06
5.82 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [13]:
# A too slow problem for a single machine... (200M points)

# use dask arrays instead of Numpy arrays for automatic graph construction
da_a = da.random.uniform(5, 10, 200_000_000)
da_b = da.random.uniform(10, 20, 200_000_000)
da_c = da.random.uniform(-0.1, 0.1, 200_000_000)

# define compute graph
output = fun(da_a, da_b, da_c)

print(output.dask)

HighLevelGraph with 15 layers.
 0. uniform-1e36026363dbc22f77b3732cf25a6949
 1. uniform-4d50f75559d799045ca46bfb83ab1510
 2. mul-7d7fa4d62baa037e6709580418ce1fc4
 3. mul-58d345cc3e88bd2084125dc92a605870
 4. mul-8d830592cff76a4198933e4a29250e26
 5. uniform-0acabcbdd9bedee7bf2458a4f535e061
 6. pow-b1f20748c9e1f7bbf9ccd6f9cec95b7a
 7. sub-70ff91ddd6b30da11501c2da556c6458
 8. sqrt-4f14dece71960fbb80641cfc437af595
 9. neg-8efce5b4394260aef1d32659776912ae
 10. add-7e5d907e2fd2f3e7cc6c47f3981e79f8
 11. truediv-1edafdc38f4b52c316fed5b93236e4d2
 12. mean_chunk-cad526579e80bd82b6d793c90b77fff3
 13. mean_combine-partial-11e1918753e621f51196f93132afd08b
 14. mean_agg-aggregate-4c622a75daac39295ecd232bcdf70137



In [14]:
%%timeit -n 1 -r 1

with (
    dask.distributed.LocalCluster(n_workers=1, threads_per_worker=10) as cluster,
    # ---
    # connect to a Dask cluster
    dask.distributed.Client(cluster) as client,
):
    # compute the result
    print("Result:", client.compute(output).result())

Result: -9.353751629793007e-06
1.55 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
